In [ ]:
# Download necessary files

#pairwise atom list used in distance calculation
!wget --no-check-certificate --content-disposition "https://uofi.box.com/shared/static/ane5qf363rb89ez62u5e4bkwaf4y4tht"
#feature filename list for umbrealla sampling trajectories
!wget --no-check-certificate --content-disposition "https://uofi.box.com/shared/static/8bxysw64rax2wix4ofa4grkdiyotloyh"
#calculated distance feature for umbrealla sampling trajectories
!wget --no-check-certificate --content-disposition "https://uofi.box.com/shared/static/9al8uf0cixepp4mggsgfz5p3qqq8v2cv"
#feature filename list for unbiased trajectories
!wget --no-check-certificate --content-disposition "https://uofi.box.com/shared/static/z1z0qm5tfda1x1q1ufrrtrnsk5np7vfg"
#calculated distance feature for unbiased trajectories
!wget --no-check-certificate --content-disposition "https://uofi.box.com/shared/static/5pyadgyf9kmf9drqwxv1kwp6thuitzyu"


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys
import pyemma

In [ ]:
#number of windows for umbrella sampling
umbrella_windows = 300
#center of umbrella windows
windows = [np.round(5 + i*0.1,1) for i in range(umbrella_windows)]
#atom pair at which bias is applied
atom_pair = [3004, 5310]
#number of states used in tram calculation
cluster = 800
#tica lag time and number of dimension used for distance feature transformation
tica_lag = 50
tica_dim = 6
# lag time used for tram
lag_time = 150

In [ ]:
#umbralla sampling distance feature filename list
biased_feature_files = pickle.load(open('CB1_classical_us_distance_feature_files.pkl','rb'))
#unbiased distance feature filename list
unbiased_feature_files = pickle.load(open('CB1_classical_distance_feature_files.pkl','rb'))

feature_files = biased_feature_files + unbiased_feature_files

In [ ]:
#pairwise atom list used in distance calculation
atom_list = pickle.load(open('CB1_classical_distance_atom_pairs.pkl','rb'))
#umbralla sampling distance feature list
biased_feature = pickle.load(open('CB1_classical_us_distance_feature.pkl','rb'))
#unbiased distance feature list
unbiased_feature = pickle.load(open('CB1_classical_distance_feature.pkl','rb'))

feature = biased_feature + unbiased_feature

In [ ]:
#bias energy calculation for each frame of all trajectories. 
#This is the bias energy each frame would feel in each ensemble.
#Every window in umbrella sampling represents a single ensemble. 
#Unbiased simulation represents another ensemble.
def biased_energy(feature,windows,index):
    bias = []
    for traj_feature in feature:
        dist = traj_feature[:,index]*10
        traj_bias = np.zeros([len(dist),len(windows)+1])
        for k in range(len(windows)):
            temp = np.array([10/(0.002*300*2)*(d-windows[k])**2 for d in dist])
            traj_bias[:,k] = temp
            traj_bias[:,-1] = [0 for d in dist] 
        bias.append(traj_bias)

    return bias

index = atom_list.index([3004, 5310])
bias =  biased_energy(feature,windows,index)

In [ ]:
#index of each trajectory belonging to a particular ensemble 
def ttrajs_calculation(feature_files,feature):
    ttrajs = []
    for i,file in enumerate(feature_files):
        traj_length = len(feature[i])
        words =  file.split('_')
        if 'us' in words:
            ttrajs.append(np.array([int(words[5])]*traj_length))
        else:
            ttrajs.append(np.array([int(300)]*traj_length))

    return ttrajs

ttrajs = ttrajs_calculation(feature_files,feature)

In [ ]:
#tica transformation of each trajectory and clustering of the tic space
def dtrajs_calculation(biased_feature,unbiased_feature,cluster):
    biased_tic = pyemma.coordinates.tica(biased_feature,lag=tica_lag,dim=tica_dim)
    data_biased_tic = biased_tic.get_output()
    transformed_biased_tic = biased_tic.transform(unbiased_feature)

    unbiased_tic = pyemma.coordinates.tica(unbiased_feature,lag=tica_lag,dim=tica_dim)
    data_unbiased_tic = unbiased_tic.get_output()
    transformed_unbiased_tic = unbiased_tic.transform(biased_feature)

    data_tic = []
    for b,d in zip(data_biased_tic,transformed_unbiased_tic):
        temp = np.concatenate((b,d),axis=1)
        data_tic.append(temp)

    for b,d in zip(transformed_biased_tic,data_unbiased_tic):
        temp = np.concatenate((b,d),axis=1)
        data_tic.append(temp)

    dtrajs = pyemma.coordinates.cluster_kmeans(data_tic,k=cluster,max_iter=100, tolerance=1e-05, stride=2).dtrajs
    return dtrajs

dtrajs = dtrajs_calculation(biased_feature,unbiased_feature,cluster)

In [ ]:
#tram building
def tram_implementation(ttrajs, dtrajs, bias,lag_time):
    ther_obj = pyemma.thermo.tram(ttrajs, dtrajs, bias, lag=lag_time, unbiased_state = 300, maxerr=1e-04, init_maxerr=1e-04)
    return ther_obj

ther_obj = tram_implementation(ttrajs, dtrajs, bias, lag_time)